In [76]:
import lusid
import lusid.models as models
from lusidjam import RefreshingToken

from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.utilities import identify_cash_items
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response
)

# Import Libraries
from datetime import datetime, timedelta, time
import pytz
import uuid
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import os
import json
import requests
from IPython.core.display import display, HTML

# pandas config
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

display(HTML("<style>.container { width:90% !important; }</style>"))

# pandas config
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)


print("LUSID Environment Initialised")
print(
    "LUSID version : ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version)

LUSID Environment Initialised
LUSID version :  0.6.7133.0


In [77]:
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
portfolios_api = api_factory.build(lusid.api.PortfoliosApi)
corporate_actions_sources_api = api_factory.build(lusid.api.CorporateActionSourcesApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

In [78]:
scope = "TakeoverExample"
fund_code = "EquityFund"

In [79]:
instrument_master = pd.read_csv("data/takeover_instruments.csv")
instrument_master = instrument_master
instrument_master

,FIGI,Ticker,Name
0,BBG000BMFD58,AMTD US,TD AMERITRADE HOLDING CORP
1,BBG000BSLZY7,SCHW US,SCHWAB (CHARLES) CORP


In [80]:
# Load the instrument into LUSID

instrument_identifier_mapping = {
    "Figi": "FIGI",
    "ClientInternal": "Ticker",
}

instrument_mapping_required = {"name": "Name"}
instrument_mapping_optional = {}

responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=instrument_master,
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type="instrument",
    identifier_mapping=instrument_identifier_mapping,
)

succ, failed, errors = format_instruments_response(responses)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,2,0,0


In [81]:
# Create the portfolio
pf_df = pd.DataFrame(
    [{"code": fund_code, "currency": "USD", "name": fund_code,}]
)

portfolio_mapping = {
    "required": {"code": "code", "display_name": "name", "base_currency": "currency",},
    "optional": {"created": "$2019-01-01T00:00:00+00:00"},
}

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=pf_df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
)

succ, failed = format_portfolios_response(result)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}]))

,success,failed
0,1,0


In [82]:
# upsert transactions
transactions = pd.read_csv("data/takeover_transactions.csv")
transactions

,portfolio_code,transaction_id,figi,ticker,name,transaction_type,transaction_date,settlement_date,transaction_units,transaction_price,total_consideration,transaction_currency
0,EquityFund,TX001,BBG000BMFD58,AMTD US,TD AMERITRADE HOLDING CORP,StockIn,10/10/2019,11/10/2019,35000,34.49,"1,207,150.00",USD


In [83]:
# map the transactions file

transaction_field_mapping_required = {
    "code": "portfolio_code",
    "transaction_id": "transaction_id",
    "type": "transaction_type",
    "transaction_date": "transaction_date",
    "settlement_date": "settlement_date",
    "units": "transaction_units",
    "transaction_price.price": "transaction_price",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "total_consideration",
    "total_consideration.currency": "transaction_currency",
    "transaction_currency": "transaction_currency",
}


transaction_identifier_mapping = {
    "Figi": "figi",
    "ClientInternal": "ticker",
}



In [84]:
responses = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=transactions,
    mapping_required=transaction_field_mapping_required,
    mapping_optional={},
    identifier_mapping=transaction_identifier_mapping,
    file_type="transaction",
)

succ, failed = format_transactions_response(responses)
display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}]))

,success,failed
0,1,0


In [85]:
# Define details for the corporate action.
instrument_name = "TD AMERITRADE HOLDING CORP"
instrument_original_figi = "BBG000BMFD58"
instrument_updated_figi = "BBG000BSLZY7"
effective_at_date = datetime(2020, 10, 6, tzinfo=pytz.utc)
corporate_action_source_code = "takeover-example-source"
corporate_action_code = "takeover-corporate-action"

In [86]:
# Create a corporate actions source.
corporate_action_source = models.CreateCorporateActionSourceRequest(
            scope=scope,
            code=corporate_action_source_code,
            display_name=corporate_action_source_code,
            description="Takeover Example Corporate Actions Source",
        )

try: 
    corporate_actions_sources_api.create_corporate_action_source(
            create_corporate_action_source_request=corporate_action_source
        )
except: 
    print("Already exists")

Already exists


In [87]:
# Apply the corporate actions source to the transaction portfolio.
transaction_portfolios_api.upsert_portfolio_details(
    scope=scope,
        code=fund_code,
        effective_at=effective_at_date,
        create_portfolio_details=models.CreatePortfolioDetails(
            corporate_action_source_id=models.ResourceId(
                scope=scope,
                code=corporate_action_source_code,
            )
        ),
)

{'base_currency': 'USD',
 'corporate_action_source_id': {'code': 'takeover-example-source',
                                'scope': 'TakeoverExample'},
 'href': 'https://lorenzdec.lusid.com/api/api/transactionportfolios/TakeoverExample/EquityFund/details?effectiveAt=2019-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2021-06-01T14%3A49%3A47.5708740%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://lorenzdec.lusid.com/api/api/portfolios/TakeoverExample/EquityFund?effectiveAt=2019-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2021-06-01T14%3A49%3A47.5708740%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://lorenzdec.lusid.com/app/insights/logs/0HM94R4S1P06K:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'origin_portfolio_id': {'code': 'EquityFund', 's

In [88]:
# Create a transition which applies to the original instrument above
transition_in = models.CorporateActionTransitionComponentRequest(
    instrument_identifiers={
        "Instrument/default/Figi": instrument_original_figi
    },
    cost_factor=1,
    units_factor=1,
)

# and has the effect of changing its FIGI to the updated FIGI and shares
rename_figi_transition = models.CorporateActionTransitionComponentRequest(
    instrument_identifiers={
        "Instrument/default/Figi": instrument_updated_figi
    },
    cost_factor=1,
    units_factor=1.0837,
)

#   while zeroing the original instrument's position.
zero_previous_position_transition = (
    models.CorporateActionTransitionComponentRequest(
        instrument_identifiers={
            "Instrument/default/Figi": instrument_original_figi
        },
        cost_factor=0,
        units_factor=0,
    )
)

# The effect of the corporate action is the transition which
# combines the input transition and the output transitions.
transition = models.CorporateActionTransitionRequest(
    input_transition=transition_in,
    output_transitions=[
        rename_figi_transition,
        zero_previous_position_transition,
    ],
)

# Create a request to upsert a corporate action with the transition above.
corporate_action_request = models.UpsertCorporateActionRequest(
    corporate_action_code=corporate_action_code,
    announcement_date=effective_at_date,
    ex_date=effective_at_date,
    record_date=effective_at_date,
    payment_date=effective_at_date,
    transitions=[transition],
)

In [89]:
# Make the request through the CorporateActionSourcesApi.
upsert_corp_act_response = corporate_actions_sources_api.batch_upsert_corporate_actions(
    scope=scope,
    code=corporate_action_source_code,
    upsert_corporate_action_request=[corporate_action_request],
)

# Quotes

In [90]:
quotes_df = pd.read_csv("data/takeover_quotes.csv")
quotes_df.head(1)

,date,figi,close_price,currency
0,10/10/2019,BBG000BMFD58,34.49,USD


In [91]:
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$Figi",
    "quote_id.effective_at": "date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "figi",
    "metric_value.unit": "currency",
    "quote_id.quote_series_id.field": "$mid",
    "metric_value.value": "close_price",
}

In [92]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope="ExampleQuotes",
    data_frame=quotes_df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes",
)

succ, failed, errors = format_quotes_response(result)
display(
    pd.DataFrame(
        data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
    )
)

,success,failed,errors
0,411,0,0


In [93]:
# Create a recipe to perform a valuation

data_scope = "ExampleQuotes"
recipe_code = "TakeOverRecipe"

configuration_recipe = models.ConfigurationRecipe(
        scope="ExampleRecipe",
        code="TakeOverRecipe",
        market=models.MarketContext(
            market_rules=[
                models.MarketDataKeyRule(
                    key="Equity.Figi.*",
                    supplier="Lusid",
                    data_scope=data_scope,
                    quote_type="Price",
                    field="mid",
                ),
            ],
            suppliers=models.MarketContextSuppliers(
                commodity="Lusid",
                credit="Lusid",
                equity="Lusid",
                fx="Lusid",
                rates="Lusid",
            ),
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="Figi",
                default_scope=scope,
                attempt_to_infer_missing_fx=True,
            ),
        ),
    )

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=models.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

print(f"Uploaded recipe: {recipe_code}")

Uploaded recipe: TakeOverRecipe
